<a href="https://colab.research.google.com/github/JoshOlu/data602/blob/main/svmHw.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
########## QUESTION A ##########

import pandas as pd
import numpy as np

from google.colab import files
uploaded = files.upload()

auto_data = pd.read_csv("Auto.csv")

auto_data.replace('?', np.nan, inplace=True)
auto_data = auto_data.apply(pd.to_numeric, errors='coerce')
auto_data.fillna(auto_data.mean(), inplace=True)

mpg_median = auto_data['mpg'].median()
auto_data['mpg_above_median'] = (auto_data['mpg'] > mpg_median).astype(int)

print(auto_data[['mpg', 'mpg_above_median']].head())


Saving Auto.csv to Auto.csv
    mpg  mpg_above_median
0  18.0                 0
1  15.0                 0
2  18.0                 0
3  16.0                 0
4  17.0                 0


In [2]:
########## QUESTION B ##########

from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC

X = auto_data.drop(columns=['mpg', 'mpg_above_median'])
y = auto_data['mpg_above_median']

X = X.loc[:, X.var() != 0]
X = X.clip(lower=X.quantile(0.01), upper=X.quantile(0.99), axis=1)

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

X_scaled = np.nan_to_num(X_scaled)

if np.isnan(X_scaled).any() or np.isinf(X_scaled).any():
    print("Warning: Numerical issues still detected in X_scaled after cleaning.")
    X_scaled = np.nan_to_num(X_scaled)

C_values = [0.01, 0.1, 1, 10, 100]
f1_scores = []

for C in C_values:
    svc = SVC(kernel='linear', C=C, random_state=42)
    cv_scores = cross_val_score(svc, X_scaled, y, cv=5, scoring='f1', error_score='raise')
    mean_f1_score = np.mean(cv_scores)
    f1_scores.append(mean_f1_score)
    print(f"C = {C}: Mean F1-Score = {mean_f1_score:.4f}")

best_C = C_values[np.argmax(f1_scores)]
print(f"\nBest C value: {best_C} with F1-Score = {max(f1_scores):.4f}")

print("Minimum values:\n", X.min())
print("Maximum values:\n", X.max())


/usr/local/lib/python3.10/dist-packages/sklearn/utils/extmath.py:1137: RuntimeWarning: invalid value encountered in divide
  updated_mean = (last_sum + new_sum) / updated_sample_count
/usr/local/lib/python3.10/dist-packages/sklearn/utils/extmath.py:1142: RuntimeWarning: invalid value encountered in divide
  T = new_sum / new_sample_count
/usr/local/lib/python3.10/dist-packages/sklearn/utils/extmath.py:1162: RuntimeWarning: invalid value encountered in divide
  new_unnormalized_variance -= correction**2 / new_sample_count


C = 0.01: Mean F1-Score = 0.8920
C = 0.1: Mean F1-Score = 0.8219
C = 1: Mean F1-Score = 0.8008
C = 10: Mean F1-Score = 0.8293
C = 100: Mean F1-Score = 0.8377

Best C value: 0.01 with F1-Score = 0.8920
Minimum values:
 cylinders          3.96
displacement      70.96
horsepower        48.00
weight          1772.48
acceleration       9.48
year              70.00
origin             1.00
name                NaN
dtype: float64
Maximum values:
 cylinders          8.000
displacement     440.560
horsepower       220.200
weight          4951.040
acceleration      22.252
year              82.000
origin             3.000
name                 NaN
dtype: float64


Based on my results, it seems that the Mean F1 scores decrease as they approach 1, then increase as they approach 100. The C value with the highest F1 score is C = 0.01, with a F1 score of 0.8920. With this score and a small C value of 0.01, it suggests that the best results come from allowing a wider margin with some misclassifications in order to improve generalization.

In [3]:
########## QUESTION C ##########

from sklearn.model_selection import cross_val_score

C_values = [0.01, 0.1, 1, 10, 100]
gamma_values = [0.001, 0.01, 0.1, 1]
degree_values = [2, 3, 4]

rbf_results = []
poly_results = []

print("RBF Kernel Results:")
for C in C_values:
    for gamma in gamma_values:
        svc_rbf = SVC(kernel='rbf', C=C, gamma=gamma)
        cv_scores = cross_val_score(svc_rbf, X_scaled, y, cv=5, scoring='f1')
        mean_f1_score = np.mean(cv_scores)
        rbf_results.append((C, gamma, mean_f1_score))
        print(f"C = {C}, gamma = {gamma}: Mean F1-Score = {mean_f1_score:.4f}")

best_rbf = max(rbf_results, key=lambda x: x[2])
print(f"\nBest RBF (C, gamma): (C={best_rbf[0]}, gamma={best_rbf[1]}) with F1-Score = {best_rbf[2]:.4f}\n")

print("Polynomial Kernel Results:")
for C in C_values:
    for degree in degree_values:
        svc_poly = SVC(kernel='poly', C=C, degree=degree)
        cv_scores = cross_val_score(svc_poly, X_scaled, y, cv=5, scoring='f1')
        mean_f1_score = np.mean(cv_scores)
        poly_results.append((C, degree, mean_f1_score))
        print(f"C = {C}, degree = {degree}: Mean F1-Score = {mean_f1_score:.4f}")

best_poly = max(poly_results, key=lambda x: x[2])
print(f"\nBest Polynomial (C, degree): (C={best_poly[0]}, degree={best_poly[1]}) with F1-Score = {best_poly[2]:.4f}")


RBF Kernel Results:
C = 0.01, gamma = 0.001: Mean F1-Score = 0.0000
C = 0.01, gamma = 0.01: Mean F1-Score = 0.0000
C = 0.01, gamma = 0.1: Mean F1-Score = 0.7007
C = 0.01, gamma = 1: Mean F1-Score = 0.0000
C = 0.1, gamma = 0.001: Mean F1-Score = 0.0000
C = 0.1, gamma = 0.01: Mean F1-Score = 0.8843
C = 0.1, gamma = 0.1: Mean F1-Score = 0.8916
C = 0.1, gamma = 1: Mean F1-Score = 0.8352
C = 1, gamma = 0.001: Mean F1-Score = 0.8843
C = 1, gamma = 0.01: Mean F1-Score = 0.8834
C = 1, gamma = 0.1: Mean F1-Score = 0.8192
C = 1, gamma = 1: Mean F1-Score = 0.8092
C = 10, gamma = 0.001: Mean F1-Score = 0.8881
C = 10, gamma = 0.01: Mean F1-Score = 0.8038
C = 10, gamma = 0.1: Mean F1-Score = 0.7814
C = 10, gamma = 1: Mean F1-Score = 0.7815
C = 100, gamma = 0.001: Mean F1-Score = 0.8154
C = 100, gamma = 0.01: Mean F1-Score = 0.8115
C = 100, gamma = 0.1: Mean F1-Score = 0.7746
C = 100, gamma = 1: Mean F1-Score = 0.7840

Best RBF (C, gamma): (C=0.1, gamma=0.1) with F1-Score = 0.8916

Polynomial Kernel 

For RBF Kernel results, a C value of 0.01 seems to have the worst F1 scores. Majority of the scores are 0, which indicates poor performance for the classifier. With a C value of 0.01 and gamma as 1, the resulting F1 score is 0, which suggests that overfitting may have happened. A C value of 1 has the best average F1 score out of all the RBF Kernel results, but the highest F1 score of 0.8916 comes from a C value of 0.1, when the gamma is 0.1. It also seems that the best F1 scores are reached when the gamma is 0.1 or less.

For the Polynomial Kernal results, a C value of 10, with a degree of 3 had the best F1 score of 0.8187. Although higher degrees to lead to overfitting, this polynomial combination was able to avoid it.

Comparing RBF and Polynomial Kernals, it seems that RBF has the better results.